In [13]:
import sys
from pyspark.sql import SparkSession

print("PySpark Script: ", sys.argv[0])

# Create a spark context and print some information about the context object
spark: SparkSession = SparkSession.builder.appName("Demo").getOrCreate()
sc = spark.sparkContext

print(sc.version)
print(sc.pythonVer)
print(sc.master)

PySpark Script:  /Users/alexbindas/anaconda3/envs/cs777-termpaper/lib/python3.12/site-packages/ipykernel_launcher.py
3.5.1
3.12
local[*]


In [15]:
# Stop Pyspark
spark.stop()
print("Spark Successfully Stopped!")

Spark Successfully Stopped!
